In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

#Just replace here to update the source.
source = 'trumpsmall'

# load ascii text and covert to lowercase
filename =  str(source)+".txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)

print ("Total Patterns: ", n_patterns)

# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

# normalize
X = X / float(n_vocab)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)

# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')


# define the checkpoint
filepath="checkpoint-{epoch:02d}-{loss:.4f}-"+ source +".hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]


C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Total Characters:  447851
Total Vocab:  35
Total Patterns:  447751


In [2]:
model.fit(X, y, epochs=200, batch_size=64, callbacks=callbacks_list)


Epoch 1/200
447751/447751 [==============================] - 4559s 10ms/step - loss: 2.5075

Epoch 00001: loss improved from inf to 2.50745, saving model to checkpoint-01-2.5075-trumpsmall.hdf5
Epoch 2/200
447751/447751 [==============================] - 4775s 11ms/step - loss: 2.0183

Epoch 00002: loss improved from 2.50745 to 2.01833, saving model to checkpoint-02-2.0183-trumpsmall.hdf5
Epoch 3/200
447751/447751 [==============================] - 4953s 11ms/step - loss: 1.8313

Epoch 00003: loss improved from 2.01833 to 1.83126, saving model to checkpoint-03-1.8313-trumpsmall.hdf5
Epoch 4/200
447751/447751 [==============================] - 6956s 16ms/step - loss: 1.7307

Epoch 00004: loss improved from 1.83126 to 1.73067, saving model to checkpoint-04-1.7307-trumpsmall.hdf5
Epoch 5/200
145536/447751 [========>.....................] - ETA: 1:18:35 - loss: 1.6669

KeyboardInterrupt: 